# Home Credit Prediction: Data Cleaning of Bureau Table


For an up-to-date version / full view of the plotly - plots, please, go to

Data Cleaning - Bureau:  https://drive.google.com/file/d/16oY7SJ5Gup31-BsuykAEf6UxrKtMOjy-/view?usp=sharing

List of all notebooks and resources for this project: https://drive.google.com/file/d/1Z8vPNZAcivWOxeh3UKFfeARbQCMkQ_NR/view?usp=sharing

## Import Modules

In [1]:
%%capture
#! pip install -q pingouin
! pip install -q scikit-optimize

In [2]:
import numpy as np
import pandas as pd

import sys
import os
import warnings
from importlib import reload

from dask import dataframe as dd
#import matplotlib.pyplot as plt
#import seaborn as sns
import plotly.express as px

##from statsmodels.stats.proportion import proportions_ztest
#import pingouin as pg

from google.colab import drive
drive.mount("/content/gdrive")

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_rows')

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Mounted at /content/gdrive


In [3]:
home_folder = '/content/gdrive/MyDrive/Colab Notebooks/Portfolio/ML_HomeCredit_DefaultRiskEvaluation/'

### Functions

The Python-file with the functions is at
https://drive.google.com/file/d/17IchsTGy2QI9sq0LTIvGvxAk2mrWs4Xz/view?usp=sharing

In [4]:
%load_ext autoreload
%autoreload 2

sys.path.append(home_folder)
import driskfunc as dfunc

# 1. Load and Update Data

data source: https://storage.googleapis.com/341-home-credit-default/home-credit-default-risk.zip

description: https://storage.googleapis.com/341-home-credit-default/Home%20Credit%20Default%20Risk.pdf

In [5]:
HCdescr = pd.read_csv(home_folder+'data/HomeCredit_columns_description.csv', encoding='latin1') #, dtype=dtype)


In [6]:
HCdescr.loc[HCdescr.Table == 'bureau.csv']

,Unnamed: 0,Table,Row,Description,Special
122,125,bureau.csv,SK_ID_CURR,"ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau",hashed
123,126,bureau.csv,SK_BUREAU_ID,Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application),hashed
124,127,bureau.csv,CREDIT_ACTIVE,Status of the Credit Bureau (CB) reported credits,NaN
125,128,bureau.csv,CREDIT_CURRENCY,Recoded currency of the Credit Bureau credit,recoded
126,129,bureau.csv,DAYS_CREDIT,How many days before current application did client apply for Credit Bureau credit,time only relative to the application
127,130,bureau.csv,CREDIT_DAY_OVERDUE,Number of days past due on CB credit at the time of application for related loan in our sample,NaN
128,131,bureau.csv,DAYS_CREDIT_ENDDATE,Remaining duration of CB credit (in days) at the time of application in Home Credit,time only relative to the application
129,132,bureau.csv,DAYS_ENDDATE_FACT,Days since CB credit ended at the time of application in Home Credit (only for closed credit),time only relative to the application
130,133,bureau.csv,AMT_CREDIT_MAX_OVERDUE,Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample),NaN
131,134,bureau.csv,CNT_CREDIT_PROLONG,How many times was the Credit Bureau credit prolonged,NaN


In [7]:
HCapp_bureau = dd.read_csv(home_folder+'data/bureau.csv')
df = HCapp_bureau
df_name = 'HCapp bureau'

In [8]:
df.npartitions

2

In [9]:
df.head(5)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [10]:
df.tail(5)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
858560,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
858561,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
858562,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
858563,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN
858564,246829,5057778,Closed,currency 1,-463,0,NaN,-387.0,NaN,0,22500.00,0.0,NaN,0.0,Microloan,-387,NaN


In [11]:
size_df = [df.shape[0].compute(),  df.shape[1]]
print('The', df_name, 'dataset has', size_df[0], 'rows and', size_df[1], 'features.')

The HCapp bureau dataset has 1716428 rows and 17 features.


# 2. Data Cleaning

* Handling missing values.
* Removing duplicate samples and features.
* Remove unneccessary columns/rows.
* Treating (here rather checking) the outliers.

## Check Missing Values and Duplicates

Overview of amounts of Nan and of data type:

In [12]:
dfunc.count_dtypes(df, name = df_name)


The dataset HCapp bureau has:
8 features of type float64.
6 features of type int64.
3 features of type string.


In [13]:
%%time
%reload_ext autoreload

nan_overview_df = dfunc.nan_type_overview_dd(df, size_df[0])
nan_overview_df_round = nan_overview_df.round(1)
nan_overview_df_round.style.background_gradient(cmap="Blues")

CPU times: user 5.11 s, sys: 788 ms, total: 5.9 s
Wall time: 5.94 s


,type,NaN[abs],NaN[%]
SK_ID_CURR,int64,0,0.000000
SK_ID_BUREAU,int64,0,0.000000
CREDIT_ACTIVE,string,0,0.000000
CREDIT_CURRENCY,string,0,0.000000
DAYS_CREDIT,int64,0,0.000000
CREDIT_DAY_OVERDUE,int64,0,0.000000
DAYS_CREDIT_ENDDATE,float64,105553,6.100000
DAYS_ENDDATE_FACT,float64,633653,36.900000
AMT_CREDIT_MAX_OVERDUE,float64,1124488,65.500000
CNT_CREDIT_PROLONG,int64,0,0.000000


### Drop Missing Value Columns

Drop all columns with too many missing values:

In [14]:
#cut_off_percentage = 70.
#
#too_many_nan_acc = nan_overview_acc['NaN[%]'
#                ].loc[nan_overview_acc['NaN[%]']
#                > cut_off_percentage].round(1)
#
#len(too_many_nan_acc), too_many_nan_acc.index.to_list()

In [15]:
#LCacc_dd = LCacc_dd.drop(columns=too_many_nan_acc.index.to_list())


In [16]:
#%time
#%reload_ext autoreload
#
#nan_overview_acc_drop = lfunc.nan_type_overview_dd(LCacc_dd, size_acc[0])
#nan_overview_acc_drop.round(1).style.background_gradient(cmap="Blues")

### Duplicates Check

In [17]:
%reload_ext autoreload

df_dup = dfunc.get_dup_dd(df, name= df_name, size=size_df[0])

Total number of duplicates in " HCapp bureau " : 0 ( 0.0 %).


## Other Checks and Modifications

#### Overview

In [18]:
df.describe().compute().T.round(1)

,count,mean,std,min,25%,50%,75%,max
SK_ID_CURR,1716428.0,278214.9,102938.6,100001.0,188947.0,278468.0,367572.0,456255.0
SK_ID_BUREAU,1716428.0,5924434.5,532265.7,5000000.0,5467037.0,5945689.0,6426782.0,6843457.0
DAYS_CREDIT,1716428.0,-1142.1,795.2,-2922.0,-1653.0,-980.0,-470.0,0.0
CREDIT_DAY_OVERDUE,1716428.0,0.8,36.5,0.0,0.0,0.0,0.0,2792.0
DAYS_CREDIT_ENDDATE,1610875.0,510.5,4994.2,-42060.0,-1122.0,-318.0,484.0,31199.0
DAYS_ENDDATE_FACT,1082775.0,-1017.4,714.0,-42023.0,-1475.0,-889.0,-420.0,0.0
AMT_CREDIT_MAX_OVERDUE,591940.0,3825.4,206031.6,0.0,0.0,0.0,0.0,115987185.0
CNT_CREDIT_PROLONG,1716428.0,0.0,0.1,0.0,0.0,0.0,0.0,9.0
AMT_CREDIT_SUM,1716415.0,354994.6,1149811.3,0.0,51813.0,126000.0,315000.0,585000000.0
AMT_CREDIT_SUM_DEBT,1458759.0,137085.1,677401.1,-4705600.3,0.0,0.0,42634.7,170100000.0


In [19]:
df_obj = df.describe(exclude=np.number).compute().T
df_obj['freq[%]'] = (df_obj['freq'].astype('float')/df_obj['count'].astype('float')*100)
df_obj.round(1)

,unique,count,top,freq,freq[%]
CREDIT_ACTIVE,4,1716428,Closed,1079273,62.9
CREDIT_CURRENCY,4,1716428,currency 1,1715020,99.9
CREDIT_TYPE,15,1716428,Consumer credit,1251615,72.9


#### Puzzling DAYS_CREDIT_ENDDATE values

The minimum values for DAYS_CREDIT_ENDDATE and DAYS_ENDDATE_FACT are lower than the earliest application date DAYS_CREDIT:

In [20]:
print('Oldest credit end date [years]:',
      round(-1*df['DAYS_CREDIT_ENDDATE'].min().compute()/365, 1))
print('Longest time since a previous credit application [years]:',
      round(-1*df['DAYS_CREDIT'].min().compute()/365, 1))

Oldest credit end date [years]: 115.2
Longest time since a previous credit application [years]: 8.0


In [21]:
df.loc[df['DAYS_ENDDATE_FACT']<-30*365].head(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
198346,104463,6550986,Closed,currency 1,-2340,0,-1836.0,-42023.0,0.0,0,225000.0,0.0,225000.0,0.0,Credit card,-2196,NaN


In [22]:
df.loc[df['DAYS_CREDIT_ENDDATE']<-30*365].head(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
55561,324038,5233073,Active,currency 1,-841,0,-16322.0,NaN,0.00,0,0.0,264909.24,0.00,0.0,Credit card,-39,NaN
67507,147912,5007091,Active,currency 1,-388,0,-41847.0,NaN,0.00,0,225000.0,0.00,225000.00,0.0,Credit card,-241,NaN
98826,200629,6540711,Closed,currency 1,-2317,0,-41824.0,-428.0,8271.99,0,0.0,0.00,0.00,0.0,Credit card,-428,NaN
106353,120334,6585564,Closed,currency 1,-2579,0,-41836.0,-687.0,0.00,0,0.0,0.00,0.00,0.0,Credit card,-687,NaN
132479,149469,5225714,Active,currency 1,-2571,0,-41862.0,NaN,0.00,0,202500.0,172003.68,30496.32,0.0,Credit card,-212,NaN
133939,381097,5227561,Closed,currency 1,-362,0,-41877.0,-301.0,0.00,0,231300.0,0.00,0.00,0.0,Consumer credit,-274,NaN
138749,109425,6555464,Closed,currency 1,-1482,0,-41873.0,-516.0,0.00,0,0.0,0.00,0.00,0.0,Credit card,-516,NaN
151269,434135,6588997,Closed,currency 1,-2876,0,-41865.0,-553.0,0.00,0,0.0,0.00,0.00,0.0,Credit card,-553,NaN
172849,402529,5594582,Active,currency 1,-407,0,-41865.0,NaN,0.00,0,225000.0,0.00,225000.00,0.0,Credit card,-320,NaN
185748,406781,6784735,Closed,currency 1,-2878,0,-41867.0,-718.0,0.00,0,0.0,0.00,0.00,0.0,Credit card,-718,NaN


In [23]:
fig = px.box(df.compute(), x="DAYS_CREDIT_ENDDATE")
fig.show()

Output hidden; open in https://colab.research.google.com to view.

These special values concern the types Credit_card,
Consumer_credit and comprise 146 rows.

Either DAYS_CREDIT_ENDDATE or DAYS_ENDDATE_FACT are affected but not both in the same row, i.e. we could use one to replace the erroneous value of the other.

In [24]:
time_limit = -30*365


df['DAYS_CREDIT_ENDDATE_mod'] = df['DAYS_CREDIT_ENDDATE']
df['DAYS_ENDDATE_FACT_mod'] = df['DAYS_ENDDATE_FACT']

df_enddate = df['DAYS_CREDIT_ENDDATE']
df_enddate_fact = df['DAYS_ENDDATE_FACT']

# take high values rows out of main table, later concat:
df_high_enddate = df.loc[(df_enddate < time_limit)]
df_high_enddate_fact = df.loc[(df_enddate_fact < time_limit)]

df = df.loc[~(df_enddate_fact < time_limit)]
df = df.loc[~(df_enddate < time_limit)]

# replace high values with a more suitable one
df_high_enddate['DAYS_CREDIT_ENDDATE_mod'] = df_high_enddate[['DAYS_CREDIT_ENDDATE',
                                                              'DAYS_ENDDATE_FACT']
                                                             ].max(axis=1)

df_high_enddate_fact['DAYS_ENDDATE_FACT_mod'] = df_high_enddate_fact[['DAYS_CREDIT_ENDDATE',
                                        'DAYS_ENDDATE_FACT']
                                       ].max(axis=1)

# put everything back together
df_reconstructed = dd.concat([df, df_high_enddate, df_high_enddate_fact], axis=0)

df = df_reconstructed

#### planned and actual credit period

In [25]:
df['TERM_PLANNED_months'] = ((df['DAYS_CREDIT_ENDDATE_mod'] - df['DAYS_CREDIT'])/365*12).round(1)

df['TERM_ACTUAL_months'] = ((df['DAYS_ENDDATE_FACT_mod'] - df['DAYS_CREDIT'])/365*12).round(1)

df['TERM_change_months'] = df['TERM_PLANNED_months'] - df['TERM_ACTUAL_months']

df['TERM_change_months'] = df['TERM_change_months'].fillna(0.0)

In [26]:
df.loc[(df.DAYS_CREDIT_ENDDATE<-30*365)][['DAYS_CREDIT',
                              'DAYS_CREDIT_ENDDATE',
                              'DAYS_CREDIT_ENDDATE_mod',
                              'DAYS_ENDDATE_FACT',
                              'DAYS_ENDDATE_FACT_mod',
                              'TERM_PLANNED_months',
                              'TERM_ACTUAL_months',
                              'TERM_change_months'
                              ]].compute().head()

,DAYS_CREDIT,DAYS_CREDIT_ENDDATE,DAYS_CREDIT_ENDDATE_mod,DAYS_ENDDATE_FACT,DAYS_ENDDATE_FACT_mod,TERM_PLANNED_months,TERM_ACTUAL_months,TERM_change_months
55561,-841,-16322.0,-16322.0,NaN,NaN,-509.0,NaN,0.0
67507,-388,-41847.0,-41847.0,NaN,NaN,-1363.0,NaN,0.0
98826,-2317,-41824.0,-428.0,-428.0,-428.0,62.1,62.1,0.0
106353,-2579,-41836.0,-687.0,-687.0,-687.0,62.2,62.2,0.0
132479,-2571,-41862.0,-41862.0,NaN,NaN,-1291.8,NaN,0.0


In [27]:
df = df.drop(columns=['DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT'])

#### interest rate

In [28]:
#df['AMT_ANNUITY_plain'] = df['AMT_CREDIT']/(df['CNT_PAYMENT'] + 0.0001)
#df['AMT_ANNUITY_diff'] = (df['AMT_ANNUITY'] - df['AMT_ANNUITY_plain'])
#df['int_rate_plain'] = df['AMT_ANNUITY_diff']/(df['AMT_ANNUITY'] + 0.0001)
#
##df['int_rate_plain_prev']  = df['int_rate_plain'] - df['RATE_INTEREST_PRIMARY'] + df['RATE_DOWN_PAYMENT']*df['int_rate_plain']
#df['AMT_ANNUITY_CREDIT_ratio'] = df['AMT_ANNUITY']/(df['AMT_CREDIT'] + 0.0001)
#df['AMT_ANNUITY_plain_ratio'] = df['AMT_ANNUITY_plain']/(df['AMT_ANNUITY'] + 0.0001)
#df = df.replace([np.inf, -np.inf], np.nan)
#

In [29]:
df['AMT_ANNUITY_plain'] = df['AMT_CREDIT_SUM_DEBT']/df['TERM_PLANNED_months']
df['int_rate'] = (df['AMT_ANNUITY'] - df['AMT_ANNUITY_plain'])/df['AMT_ANNUITY_plain']
df['AMT_ANNUITY_CREDIT_ratio'] = df['AMT_ANNUITY']/df['AMT_CREDIT_SUM_DEBT']
df['AMT_CREDIT_SUM_debt_limit'] = ((df['AMT_CREDIT_SUM_LIMIT'] -
                                    df['AMT_CREDIT_SUM_DEBT']) #+
                                   #df['AMT_CREDIT_SUM']
                                   )/df['TERM_PLANNED_months']

df['AMT_CREDIT_DEBT_LIMIT_RATIO']  = df['AMT_CREDIT_SUM_DEBT']/df['AMT_CREDIT_SUM_LIMIT']
df['AMT_CREDIT_DEBT_SUM_RATIO']  = df['AMT_CREDIT_SUM_DEBT']/df['AMT_CREDIT_SUM']
df = df.replace([np.inf, -np.inf], np.nan)


In [30]:
df.loc[(df.AMT_ANNUITY>0)&
       (df.AMT_CREDIT_SUM_DEBT>0)&
       (df.AMT_CREDIT_SUM_LIMIT>0)&
        (df.AMT_ANNUITY_plain>0)][[
            'int_rate', 'AMT_ANNUITY',  'AMT_ANNUITY_plain',
            'AMT_ANNUITY_CREDIT_ratio', 'AMT_CREDIT_SUM_debt_limit',
            'AMT_CREDIT_DEBT_LIMIT_RATIO', 'AMT_CREDIT_DEBT_SUM_RATIO',
            'CREDIT_ACTIVE'
                              ]].compute().head(20).round(3)

,int_rate,AMT_ANNUITY,AMT_ANNUITY_plain,AMT_ANNUITY_CREDIT_ratio,AMT_CREDIT_SUM_debt_limit,AMT_CREDIT_DEBT_LIMIT_RATIO,AMT_CREDIT_DEBT_SUM_RATIO,CREDIT_ACTIVE
983,7.796,58519.215,6652.737,0.146,-5842.787,8.214,0.891,Active
2288,6.155,8460.000,1182.418,0.198,-1118.299,18.441,0.949,Active
2345,184.944,4500.000,24.201,0.204,-23.657,44.529,0.978,Active
3345,79.013,4500.000,56.241,0.089,-12.913,1.298,0.565,Active
3492,2.793,4500.000,1186.285,0.067,6197.376,0.161,0.138,Active
3499,6.420,1800.000,242.588,0.200,0.000,1.000,0.500,Active
3535,17.067,4519.800,250.169,0.050,-225.568,10.169,0.910,Active
3541,2.836,11700.000,3050.427,0.064,-2778.706,11.226,0.918,Active
3597,8.662,4500.000,465.765,0.010,-464.755,461.227,0.998,Active
4060,2.761,5850.000,1555.343,0.063,233.921,0.869,0.456,Active


#### CNT_CREDIT_PROLONG - Binary

In [31]:
df['CNT_CREDIT_PROLONG'].value_counts().compute()

,count
CNT_CREDIT_PROLONG,
0,1707314
1,7620
2,1222
3,191
4,54
5,21
6,2
9,2
7,1


In [32]:
df['CNT_CREDIT_PROLONG_bin'] = df['CNT_CREDIT_PROLONG']
df['CNT_CREDIT_PROLONG_bin'] = df['CNT_CREDIT_PROLONG_bin'].mask(~(df['CNT_CREDIT_PROLONG_bin']==0), 1)
df['CNT_CREDIT_PROLONG_bin'].value_counts().compute()

,count
CNT_CREDIT_PROLONG_bin,
0,1707314
1,9114


#### CREDIT_DAY_OVERDUE - categories

Use categories from bureau_balance.csv

In [33]:
df_od = df['CREDIT_DAY_OVERDUE']
df_od = df_od.mask((df_od==0), 0)
df_od = df_od.mask(((df_od>=1) & (df_od<=30)), 1)
df_od = df_od.mask(((df_od>=31) & (df_od<=60)), 2)
df_od = df_od.mask(((df_od>=61) & (df_od<=90)), 3)
df_od = df_od.mask(((df_od>=91) & (df_od<=120)), 4)
df_od = df_od.mask((df_od>=121), 5)
df['CREDIT_DAY_OVERDUE_cat'] = df_od
df['CREDIT_DAY_OVERDUE_cat'].value_counts().compute()

,count
CREDIT_DAY_OVERDUE_cat,
0,1712211
1,2011
5,1143
2,692
3,240
4,131


#### CREDIT_CURRENCY - Binary

In [34]:
df.CREDIT_CURRENCY.value_counts().compute()

,count
CREDIT_CURRENCY,
currency 1,1715020
currency 2,1224
currency 3,174
currency 4,10


In [35]:
df_cur = df['CREDIT_CURRENCY']
df_cur = df_cur.mask(~(df_cur=='currency 1'), '0') # order of commands important!
df_cur = df_cur.mask((df_cur=='currency 1'), '1')
#df_cur = df_cur.replace({'0': 0, '1': 1})
df['CURRENCY_1'] = df_cur.astype('int')
df['CURRENCY_1'].value_counts().compute()

,count
CURRENCY_1,
1,1715020
0,1408


#### CREDIT_TYPE - reduce categories

In [36]:
type_counts = df.CREDIT_TYPE.value_counts().compute()
type_counts

,count
CREDIT_TYPE,
Consumer credit,1251615
Credit card,402195
Car loan,27690
Mortgage,18391
Microloan,12413
Loan for business development,1975
Another type of loan,1017
Unknown type of loan,555
Loan for working capital replenishment,469


counts > 1% could still have an affect as an own category.

Let's cut at 0.5% to keep the sum of the rest of the categories at the last rank.


In [37]:
minorities = type_counts[type_counts<size_df[0]/200].index.tolist()

In [38]:
df_type = df.CREDIT_TYPE
#df_type = df_type.mask(~(df_type=='currency 1'), 0) # order of commands important!
df_type = df_type.mask((df_type.isin(minorities)), 'Other loan types')
df['CREDIT_TYPE_red'] = df_type
df['CREDIT_TYPE_red'].value_counts().compute()/size_df[0]

,count
CREDIT_TYPE_red,
Consumer credit,0.729197
Credit card,0.234321
Car loan,0.016132
Mortgage,0.010715
Microloan,0.007232
Other loan types,0.002403


Get rid of spaces in the strings (--> underscore)

In [39]:
renamed = {}
for key in df['CREDIT_TYPE_red'].unique():
    renamed[key] = key.replace(' ','_')

df['CREDIT_TYPE_red'] = df['CREDIT_TYPE_red'].replace(renamed)

In [40]:
df_type = df.categorize(columns = ['CREDIT_TYPE_red']).CREDIT_TYPE_red
df_type_cat = dd.get_dummies(df_type, prefix = 'TYPE', dtype=int)

#### CREDIT_ACTIVE - reduce categories and dummy encode

In [41]:
df_car = df['CREDIT_ACTIVE']
df_car = df_car.mask((df_car=='Sold'), 'Sold-Bad')
df_car = df_car.mask((df_car=='Bad debt'), 'Sold-Bad')
df['CREDIT_ACTIVE_red'] = df_car
df['CREDIT_ACTIVE_red'].value_counts().compute()

,count
CREDIT_ACTIVE_red,
Closed,1079273
Active,630607
Sold-Bad,6548


In [42]:
df_stat = df.categorize(columns = ['CREDIT_ACTIVE_red']).CREDIT_ACTIVE_red
df_status_cat = dd.get_dummies(df_stat, prefix = 'ACTIVITY', dtype=int)

df_stat_type = dd.concat([df_status_cat, df_type_cat], axis=1)

df_with_dummies = dd.concat([df, df_stat_type], axis=1)
#df_with_dummies = df_with_dummies.drop(columns='CREDIT_ACTIVE_red')

df_with_dummies.head(20)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DAYS_CREDIT_ENDDATE_mod,DAYS_ENDDATE_FACT_mod,TERM_PLANNED_months,TERM_ACTUAL_months,TERM_change_months,AMT_ANNUITY_plain,int_rate,AMT_ANNUITY_CREDIT_ratio,AMT_CREDIT_SUM_debt_limit,AMT_CREDIT_DEBT_LIMIT_RATIO,AMT_CREDIT_DEBT_SUM_RATIO,CNT_CREDIT_PROLONG_bin,CREDIT_DAY_OVERDUE_cat,CURRENCY_1,CREDIT_TYPE_red,CREDIT_ACTIVE_red,ACTIVITY_Active,ACTIVITY_Closed,ACTIVITY_Sold-Bad,TYPE_Car_loan,TYPE_Consumer_credit,TYPE_Credit_card,TYPE_Microloan,TYPE_Mortgage,TYPE_Other_loan_types
0,215354,5714462,Closed,currency 1,-497,0,NaN,0,91323.00,0.00,NaN,0.0,Consumer credit,-131,NaN,-153.0,-153.0,11.3,11.3,0.0,0.000000,NaN,NaN,NaN,NaN,0.000000,0,0,1,Consumer_credit,Closed,0,1,0,0,1,0,0,0,0
1,215354,5714463,Active,currency 1,-208,0,NaN,0,225000.00,171342.00,NaN,0.0,Credit card,-20,NaN,1075.0,NaN,42.2,NaN,0.0,4060.236967,NaN,NaN,NaN,NaN,0.761520,0,0,1,Credit_card,Active,1,0,0,0,0,1,0,0,0
2,215354,5714464,Active,currency 1,-203,0,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,528.0,NaN,24.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,Consumer_credit,Active,1,0,0,0,1,0,0,0,0
3,215354,5714465,Active,currency 1,-203,0,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,Credit_card,Active,1,0,0,0,0,1,0,0,0
4,215354,5714466,Active,currency 1,-629,0,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,1197.0,NaN,60.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,Consumer_credit,Active,1,0,0,0,1,0,0,0,0
5,215354,5714467,Active,currency 1,-273,0,0.0,0,180000.00,71017.38,108982.62,0.0,Credit card,-31,NaN,27460.0,NaN,911.8,NaN,0.0,77.887015,NaN,NaN,41.637684,0.651639,0.394541,0,0,1,Credit_card,Active,1,0,0,0,0,1,0,0,0
6,215354,5714468,Active,currency 1,-43,0,0.0,0,42103.80,42103.80,0.00,0.0,Consumer credit,-22,NaN,79.0,NaN,4.0,NaN,0.0,10525.950000,NaN,NaN,-10525.950000,NaN,1.000000,0,0,1,Consumer_credit,Active,1,0,0,0,1,0,0,0,0
7,162297,5714469,Closed,currency 1,-1896,0,14985.0,0,76878.45,0.00,0.00,0.0,Consumer credit,-1710,NaN,-1684.0,-1710.0,7.0,6.1,0.9,0.000000,NaN,NaN,0.000000,NaN,0.000000,0,0,1,Consumer_credit,Closed,0,1,0,0,1,0,0,0,0
8,162297,5714470,Closed,currency 1,-1146,0,0.0,0,103007.70,0.00,0.00,0.0,Consumer credit,-840,NaN,-811.0,-840.0,11.0,10.1,0.9,0.000000,NaN,NaN,0.000000,NaN,0.000000,0,0,1,Consumer_credit,Closed,0,1,0,0,1,0,0,0,0
9,162297,5714471,Active,currency 1,-1146,0,0.0,0,4500.00,0.00,0.00,0.0,Credit card,-690,NaN,-484.0,NaN,21.8,NaN,0.0,0.000000,NaN,NaN,0.000000,NaN,0.000000,0,0,1,Credit_card,Active,1,0,0,0,0,1,0,0,0


In [43]:
df_with_dummies3 = df_with_dummies.loc[df_with_dummies.DAYS_CREDIT_ENDDATE_mod>-20*365]
df_with_dummies3.describe().compute().T.round(1)

,count,mean,std,min,25%,50%,75%,max
SK_ID_CURR,1610824.0,278173.0,102940.1,100001.0,188854.5,277578.0,367486.0,456255.0
SK_ID_BUREAU,1610824.0,5923473.2,532345.4,5000000.0,5465894.0,5989962.0,6550986.0,6843457.0
DAYS_CREDIT,1610824.0,-1165.7,799.5,-2922.0,-1717.0,-1026.0,-495.0,0.0
CREDIT_DAY_OVERDUE,1610824.0,0.9,37.4,0.0,0.0,0.0,0.0,2792.0
AMT_CREDIT_MAX_OVERDUE,535126.0,4097.4,216682.4,0.0,0.0,0.0,0.0,115987185.0
CNT_CREDIT_PROLONG,1610824.0,0.0,0.1,0.0,0.0,0.0,0.0,9.0
AMT_CREDIT_SUM,1610820.0,358976.4,1160551.1,0.0,50952.9,125017.2,315000.0,585000000.0
AMT_CREDIT_SUM_DEBT,1376344.0,136583.1,679823.3,-4705600.3,0.0,0.0,29340.0,170100000.0
AMT_CREDIT_SUM_LIMIT,1064882.0,3554.0,33996.4,-586406.1,0.0,0.0,0.0,4705600.3
AMT_CREDIT_SUM_OVERDUE,1610824.0,36.9,5994.9,0.0,0.0,0.0,0.0,3756681.0


In [44]:
len(df_with_dummies)

1716428

## Merge with balance

In [45]:
HCapp_bureau_bal = dd.read_csv(home_folder+'cleaned/HC_bureau_balance_cleaned.csv')

In [46]:
df_with_dummies_bal = df_with_dummies.merge(HCapp_bureau_bal, on = 'SK_ID_BUREAU', how = 'outer')
#df_with_dummies_bal.head(5)

In [47]:
#size_merged = len(df_with_dummies_bal)
#size_merged_balance = len(df_with_dummies_bal.loc[~(df_with_dummies_bal['MONTHS_BALANCE'].isna())])
#data_ratio = round(size_merged/size_df[0]*100, 1)
#data_ratio_balance = round(size_merged_balance/size_df[0]*100, 1)
#
#
#print('Data size after merging with balance data:', size_merged)
#print('That is', data_ratio, '% of the initial dataset.')
#print(data_ratio_balance, '% of the data hold information from the "balance" table.')

Some SK_ID_BUREAU entries from "balance" are not present in the "bureau" table - drop them:

In [48]:
df_with_dummies_bal.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DAYS_CREDIT_ENDDATE_mod,DAYS_ENDDATE_FACT_mod,TERM_PLANNED_months,TERM_ACTUAL_months,TERM_change_months,AMT_ANNUITY_plain,int_rate,AMT_ANNUITY_CREDIT_ratio,AMT_CREDIT_SUM_debt_limit,AMT_CREDIT_DEBT_LIMIT_RATIO,AMT_CREDIT_DEBT_SUM_RATIO,CNT_CREDIT_PROLONG_bin,CREDIT_DAY_OVERDUE_cat,CURRENCY_1,CREDIT_TYPE_red,CREDIT_ACTIVE_red,ACTIVITY_Active,ACTIVITY_Closed,ACTIVITY_Sold-Bad,TYPE_Car_loan,TYPE_Consumer_credit,TYPE_Credit_card,TYPE_Microloan,TYPE_Mortgage,TYPE_Other_loan_types,MONTHS_BALANCE,STATUS,MONTHS_BALANCE_count,STATUS_0_mean,STATUS_1_mean,STATUS_2_mean,STATUS_3_mean,STATUS_4_mean,STATUS_5_mean,STATUS_C_mean,STATUS_X_mean
0,324038.0,5233073,Active,currency 1,-841.0,0.0,0.00,0.0,0.0,264909.240,0.000,0.0,Credit card,-39.0,NaN,-16322.0,NaN,-509.0,NaN,0.0,-520.450373,NaN,NaN,520.450373,NaN,NaN,0.0,0.0,1.0,Credit_card,Active,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,402529.0,5594582,Active,currency 1,-407.0,0.0,0.00,0.0,225000.0,0.000,225000.000,0.0,Credit card,-320.0,NaN,-41865.0,NaN,-1363.0,NaN,0.0,-0.000000,NaN,NaN,-165.077036,0.000000,0.000000,0.0,0.0,1.0,Credit_card,Active,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,406781.0,6784735,Closed,currency 1,-2878.0,0.0,0.00,0.0,0.0,0.000,0.000,0.0,Credit card,-718.0,NaN,-718.0,-718.0,71.0,71.0,0.0,0.000000,NaN,NaN,0.000000,NaN,NaN,0.0,0.0,1.0,Credit_card,Closed,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,232153.0,5051826,Closed,currency 1,-1527.0,0.0,0.00,0.0,0.0,0.000,0.000,0.0,Credit card,-606.0,NaN,-606.0,-606.0,30.3,30.3,0.0,0.000000,NaN,NaN,0.000000,NaN,NaN,0.0,0.0,1.0,Credit_card,Closed,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,362241.0,5063219,Active,currency 1,-2325.0,0.0,6853.95,0.0,135000.0,116840.295,18159.705,0.0,Credit card,-14.0,NaN,-41832.0,NaN,-1298.9,NaN,0.0,-89.953264,NaN,NaN,75.972431,6.434041,0.865484,0.0,0.0,1.0,Credit_card,Active,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
df_with_dummies_bal.loc[df_with_dummies_bal['SK_ID_CURR'].isna()].head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DAYS_CREDIT_ENDDATE_mod,DAYS_ENDDATE_FACT_mod,TERM_PLANNED_months,TERM_ACTUAL_months,TERM_change_months,AMT_ANNUITY_plain,int_rate,AMT_ANNUITY_CREDIT_ratio,AMT_CREDIT_SUM_debt_limit,AMT_CREDIT_DEBT_LIMIT_RATIO,AMT_CREDIT_DEBT_SUM_RATIO,CNT_CREDIT_PROLONG_bin,CREDIT_DAY_OVERDUE_cat,CURRENCY_1,CREDIT_TYPE_red,CREDIT_ACTIVE_red,ACTIVITY_Active,ACTIVITY_Closed,ACTIVITY_Sold-Bad,TYPE_Car_loan,TYPE_Consumer_credit,TYPE_Credit_card,TYPE_Microloan,TYPE_Mortgage,TYPE_Other_loan_types,MONTHS_BALANCE,STATUS,MONTHS_BALANCE_count,STATUS_0_mean,STATUS_1_mean,STATUS_2_mean,STATUS_3_mean,STATUS_4_mean,STATUS_5_mean,STATUS_C_mean,STATUS_X_mean
285928,NaN,5719020,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,C,97.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
285929,NaN,5719281,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,C,97.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
285930,NaN,5223354,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-21.0,C,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0.921053,0.078947
285931,NaN,5223607,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-55.0,C,42.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.000000
285932,NaN,5230569,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,C,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.896907,0.103093


In [50]:
df_with_dummies_bal = df_with_dummies_bal.loc[~(df_with_dummies_bal['SK_ID_CURR'].isna())]

In [51]:
df_with_dummies_bal.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'AMT_CREDIT_MAX_OVERDUE',
       'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT',
       'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE',
       'DAYS_CREDIT_UPDATE', 'AMT_ANNUITY', 'DAYS_CREDIT_ENDDATE_mod',
       'DAYS_ENDDATE_FACT_mod', 'TERM_PLANNED_months', 'TERM_ACTUAL_months',
       'TERM_change_months', 'AMT_ANNUITY_plain', 'int_rate',
       'AMT_ANNUITY_CREDIT_ratio', 'AMT_CREDIT_SUM_debt_limit',
       'AMT_CREDIT_DEBT_LIMIT_RATIO', 'AMT_CREDIT_DEBT_SUM_RATIO',
       'CNT_CREDIT_PROLONG_bin', 'CREDIT_DAY_OVERDUE_cat', 'CURRENCY_1',
       'CREDIT_TYPE_red', 'CREDIT_ACTIVE_red', 'ACTIVITY_Active',
       'ACTIVITY_Closed', 'ACTIVITY_Sold-Bad', 'TYPE_Car_loan',
       'TYPE_Consumer_credit', 'TYPE_Credit_card', 'TYPE_Microloan',
       'TYPE_Mortgage', 'TYPE_Other_loan_types', 'MONTHS_BALANCE', 'STATUS',
       'MONTHS_BALANCE_co

## Aggregate by SK_ID_CURR

In [52]:
df_mean = df_with_dummies_bal.groupby("SK_ID_CURR").mean(numeric_only=True)
df_mean = df_mean.drop(columns='SK_ID_BUREAU')
df_count = df_with_dummies_bal.groupby("SK_ID_CURR")['SK_ID_BUREAU'].count().to_frame()
df_count = df_count.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_count'})

# aggregate active credits = current responsibilities
active = df_with_dummies_bal.loc[df_with_dummies_bal.CREDIT_ACTIVE=='Active']

df_sums_amt = active.groupby("SK_ID_CURR")['AMT_ANNUITY',
                                           'AMT_CREDIT_MAX_OVERDUE',
                                           'AMT_CREDIT_SUM',
                                           'AMT_CREDIT_SUM_DEBT',
                                           'AMT_CREDIT_SUM_LIMIT',
                                           'AMT_CREDIT_SUM_OVERDUE'
                                           ].sum()
df_sums_amt = df_sums_amt.rename(columns={'AMT_ANNUITY': 'AMT_ANNUITY_sum_active_bureau',
                                          'AMT_CREDIT_MAX_OVERDUE': 'AMT_CREDIT_MAX_OVERDUE_sum_active',
                                          'AMT_CREDIT_SUM': 'AMT_CREDIT_SUM_sum_active',
                                          'AMT_CREDIT_SUM_DEBT': 'AMT_CREDIT_SUM_DEBT_sum_active',
                                          'AMT_CREDIT_SUM_LIMIT': 'AMT_CREDIT_SUM_LIMIT_sum_active',
                                          'AMT_CREDIT_SUM_OVERDUE': 'AMT_CREDIT_SUM_OVERDUE_sum_active'
})


df_count_active = active.groupby("SK_ID_CURR")['SK_ID_BUREAU'].count().to_frame()
df_count_active = df_count_active.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_active_count'})

df_active_final = dd.concat([df_sums_amt, df_count_active], axis=1)

df_all_final = dd.concat([df_mean, df_count], axis=1)

df_final = df_all_final.merge(df_active_final, left_index=True,
                                      right_index=True, #suffixes=('_train', '_test'),
                                      how='outer')

df_final = df_final.reset_index()
df_final.head()

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,DAYS_CREDIT_ENDDATE_mod,DAYS_ENDDATE_FACT_mod,TERM_PLANNED_months,TERM_ACTUAL_months,TERM_change_months,AMT_ANNUITY_plain,int_rate,AMT_ANNUITY_CREDIT_ratio,AMT_CREDIT_SUM_debt_limit,AMT_CREDIT_DEBT_LIMIT_RATIO,AMT_CREDIT_DEBT_SUM_RATIO,CNT_CREDIT_PROLONG_bin,CREDIT_DAY_OVERDUE_cat,CURRENCY_1,ACTIVITY_Active,ACTIVITY_Closed,ACTIVITY_Sold-Bad,TYPE_Car_loan,TYPE_Consumer_credit,TYPE_Credit_card,TYPE_Microloan,TYPE_Mortgage,TYPE_Other_loan_types,MONTHS_BALANCE,MONTHS_BALANCE_count,STATUS_0_mean,STATUS_1_mean,STATUS_2_mean,STATUS_3_mean,STATUS_4_mean,STATUS_5_mean,STATUS_C_mean,STATUS_X_mean,SK_ID_BUREAU_count,AMT_ANNUITY_sum_active_bureau,AMT_CREDIT_MAX_OVERDUE_sum_active,AMT_CREDIT_SUM_sum_active,AMT_CREDIT_SUM_DEBT_sum_active,AMT_CREDIT_SUM_LIMIT_sum_active,AMT_CREDIT_SUM_OVERDUE_sum_active,SK_ID_BUREAU_active_count
0,100001.0,-735.000000,0.0,NaN,0.0,207623.571429,85240.928571,0.00000,0.0,-93.142857,3545.357143,82.428571,-825.500000,26.871429,7.525000,3.700000,1880.426714,0.914890,0.051609,-1427.987417,NaN,0.282518,0.0,0.0,1.0,0.428571,0.571429,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,24.571429,0.336651,0.007519,0.0,0.0,0.0,0.0,0.441240,0.214590,7,24817.5,0.0,884025.000,596686.5,0.000,0.0,3.0
1,100002.0,-874.000000,0.0,1681.029,0.0,108131.945625,49156.200000,7997.14125,0.0,-499.875000,0.000000,-349.000000,-697.500000,23.666667,9.116667,5.187500,1367.729549,-1.000000,0.000000,-1367.729549,0.0,0.136545,0.0,0.0,1.0,0.250000,0.750000,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,-15.5,13.750000,0.406960,0.255682,0.0,0.0,0.0,0.0,0.175426,0.161932,8,0.0,40.5,481988.565,245781.0,31988.565,0.0,2.0
2,100003.0,-1400.750000,0.0,0.000,0.0,254350.125000,0.000000,202500.00000,0.0,-816.000000,NaN,-544.500000,-1097.333333,28.150000,18.700000,-0.850000,0.000000,NaN,NaN,3380.634391,0.0,0.000000,0.0,0.0,1.0,0.250000,0.750000,0.0,0.0,0.500000,0.500000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,0.0,0.0,810000.000,0.0,810000.000,0.0,1.0
3,100004.0,-867.000000,0.0,0.000,0.0,94518.900000,0.000000,0.00000,0.0,-532.000000,NaN,-488.500000,-532.500000,12.450000,11.000000,1.450000,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.0,0.0,1.0,0.000000,1.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100005.0,-190.666667,0.0,0.000,0.0,219042.000000,189469.500000,0.00000,0.0,-54.333333,1420.500000,439.333333,-123.000000,20.700000,8.200000,-0.033333,5178.187500,-0.495113,0.084148,-5178.187500,NaN,0.601256,0.0,0.0,1.0,0.666667,0.333333,0.0,0.0,0.666667,0.333333,0.0,0.0,0.0,0.0,7.000000,0.735043,0.000000,0.0,0.0,0.0,0.0,0.128205,0.136752,3,4261.5,0.0,598626.000,568408.5,0.000,0.0,2.0


In [53]:
size_df_final = [df_final.shape[0].compute(),  df_final.shape[1]]

print('The condensed dataset has', size_df_final[0], 'rows and', size_df_final[1], 'features.')
print('Initial size was', size_df[0], 'rows and', size_df[1], 'features.')

The condensed dataset has 305811 rows and 52 features.
Initial size was 1716428 rows and 17 features.


In [54]:
nan_overview_df = dfunc.nan_type_overview_dd(df_final, size_df_final[0])
nan_overview_df.round(1).style.background_gradient(cmap="Blues")

,type,NaN[abs],NaN[%]
SK_ID_CURR,int64,0,0.000000
DAYS_CREDIT,float64,0,0.000000
CREDIT_DAY_OVERDUE,float64,0,0.000000
AMT_CREDIT_MAX_OVERDUE,float64,92840,30.400000
CNT_CREDIT_PROLONG,float64,0,0.000000
AMT_CREDIT_SUM,float64,2,0.000000
AMT_CREDIT_SUM_DEBT,float64,8372,2.700000
AMT_CREDIT_SUM_LIMIT,float64,25308,8.300000
AMT_CREDIT_SUM_OVERDUE,float64,0,0.000000
DAYS_CREDIT_UPDATE,float64,0,0.000000


In [55]:
df_final.describe().compute().T.round(1)

,count,mean,std,min,25%,50%,75%,max
SK_ID_CURR,305811.0,278047.3,102849.6,100001.0,188878.5,277895.0,367184.5,456255.0
DAYS_CREDIT,305811.0,-1083.8,558.4,-2922.0,-1431.0,-1052.5,-670.4,0.0
CREDIT_DAY_OVERDUE,305811.0,1.0,25.0,0.0,0.0,0.0,0.0,2776.0
AMT_CREDIT_MAX_OVERDUE,212971.0,5004.9,285681.9,0.0,0.0,0.0,2063.7,115987185.0
CNT_CREDIT_PROLONG,305811.0,0.0,0.1,0.0,0.0,0.0,0.0,6.0
AMT_CREDIT_SUM,305809.0,380739.8,879286.5,0.0,103961.6,197297.2,397864.6,198072344.2
AMT_CREDIT_SUM_DEBT,297439.0,161634.1,536767.7,-1083614.7,0.0,44760.4,143600.4,51750000.0
AMT_CREDIT_SUM_LIMIT,280503.0,5898.5,29921.8,-97891.7,0.0,0.0,0.0,4500000.0
AMT_CREDIT_SUM_OVERDUE,305811.0,46.0,4956.6,0.0,0.0,0.0,0.0,1617403.5
DAYS_CREDIT_UPDATE,305811.0,-543.0,446.4,-41890.0,-773.7,-479.3,-209.0,14.0


# Export

In [56]:
%%capture
! mkdir home_folder+'cleaned/'
df_final.to_csv(home_folder+'cleaned/HC_bureau_cleaned.csv',
                 index=False, single_file = True)